# דחיסה - האפמן ו-Lempel-Ziv

מחברת אינטראקטיבית להבנת אלגוריתמי דחיסה.

## 1. קידוד האפמן (Huffman)

### ספירת תדירויות

In [ ]:
def char_count(corpus):
    """ספירת תווים בקורפוס"""
    d = {}
    for ch in corpus:
        d[ch] = d.get(ch, 0) + 1
    return d

# דוגמה
text = "abracadabra"
freq = char_count(text)
print(f"Text: '{text}'")
print(f"Frequencies: {freq}")

### בניית עץ האפמן

In [ ]:
class HuffmanNode:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right
    
    def __repr__(self):
        if self.char:
            return f"Leaf('{self.char}', {self.freq})"
        return f"Node({self.freq})"

def build_huffman_tree(freq_dict):
    """בניית עץ האפמן מתדירויות"""
    # יצירת צמתים ראשוניים
    nodes = [HuffmanNode(char=ch, freq=f) for ch, f in freq_dict.items()]
    
    # מיזוג עד שנשאר צומת אחד
    while len(nodes) > 1:
        # מיון לפי תדירות
        nodes.sort(key=lambda x: x.freq)
        # מיזוג שני הקטנים
        left = nodes.pop(0)
        right = nodes.pop(0)
        merged = HuffmanNode(freq=left.freq + right.freq, left=left, right=right)
        nodes.append(merged)
    
    return nodes[0] if nodes else None

In [ ]:
# בניית העץ
tree = build_huffman_tree(freq)
print("Root:", tree)

### יצירת קודים מהעץ

In [ ]:
def build_codes(node, prefix="", codes=None):
    """יצירת מילון קודים מעץ האפמן"""
    if codes is None:
        codes = {}
    
    if node.char is not None:  # עלה
        codes[node.char] = prefix if prefix else "0"  # מקרה של תו יחיד
    else:
        build_codes(node.left, prefix + "0", codes)
        build_codes(node.right, prefix + "1", codes)
    
    return codes

codes = build_codes(tree)
print("Huffman codes:")
for ch, code in sorted(codes.items()):
    print(f"  '{ch}': {code}")

### קידוד ופענוח

In [ ]:
def huffman_encode(text, codes):
    """קידוד טקסט"""
    return "".join(codes[ch] for ch in text)

def huffman_decode(encoded, tree):
    """פענוח טקסט"""
    result = []
    node = tree
    for bit in encoded:
        if bit == '0':
            node = node.left
        else:
            node = node.right
        
        if node.char is not None:  # הגענו לעלה
            result.append(node.char)
            node = tree
    
    return "".join(result)

# דוגמה
encoded = huffman_encode(text, codes)
decoded = huffman_decode(encoded, tree)

print(f"Original: '{text}'")
print(f"Encoded:  '{encoded}'")
print(f"Decoded:  '{decoded}'")
print(f"\nOriginal bits: {len(text) * 8}")
print(f"Encoded bits:  {len(encoded)}")
print(f"Compression:   {len(encoded) / (len(text) * 8) * 100:.1f}%")

### ויזואליזציה של העץ

In [ ]:
def print_huffman_tree(node, prefix="", is_left=True):
    if node is not None:
        print(prefix + ("|-- " if is_left else "`-- ") + 
              (f"'{node.char}'" if node.char else f"({node.freq})"))
        if node.left or node.right:
            new_prefix = prefix + ("|   " if is_left else "    ")
            if node.left:
                print_huffman_tree(node.left, new_prefix, True)
            if node.right:
                print_huffman_tree(node.right, new_prefix, False)

print("Huffman Tree:")
print_huffman_tree(tree, "", False)

## 2. Lempel-Ziv (LZ)

### מימוש בסיסי

In [ ]:
def lz_compress(text, window_size=100):
    """דחיסת LZ - מחזירה ייצוג ביניים"""
    result = []
    i = 0
    
    while i < len(text):
        best_offset = 0
        best_length = 0
        
        # חפש את ההתאמה הארוכה ביותר בחלון
        start = max(0, i - window_size)
        for j in range(start, i):
            length = 0
            while (i + length < len(text) and 
                   text[j + length] == text[i + length] and
                   j + length < i):  # לא לחרוג מהמיקום הנוכחי
                length += 1
            
            if length > best_length:
                best_length = length
                best_offset = i - j
        
        # החלט אם לדחוס או לא
        if best_length >= 3:  # כדאי לדחוס רק אם חוסכים
            result.append([best_offset, best_length])
            i += best_length
        else:
            result.append(text[i])
            i += 1
    
    return result

In [ ]:
def lz_decompress(compressed):
    """פענוח LZ"""
    result = []
    
    for item in compressed:
        if isinstance(item, str):
            result.append(item)
        else:
            offset, length = item
            start = len(result) - offset
            for i in range(length):
                result.append(result[start + i])
    
    return "".join(result)

In [ ]:
# דוגמאות
texts = [
    "abracadabra",
    "abcabcabcabc",
    "aaaaaaaaaa",
    "ABACCBACACA"
]

for text in texts:
    compressed = lz_compress(text)
    decompressed = lz_decompress(compressed)
    print(f"Original:    '{text}'")
    print(f"Compressed:  {compressed}")
    print(f"Decompressed: '{decompressed}'")
    print(f"Match: {text == decompressed}")
    print()

## 3. השוואה בין השיטות

| תכונה | האפמן | LZ |
|-------|-------|----|
| סוג | ללא אובדן | ללא אובדן |
| מבוסס על | תדירויות תווים | חזרות במחרוזת |
| דורש | קורפוס/סטטיסטיקה | לא |
| יעיל עבור | טקסט עם התפלגות לא אחידה | טקסט עם חזרות |
| שילוב | נפוץ לשלב שניהם (gzip) | נפוץ לשלב שניהם |

## 4. חישוב ביטים

### מתי כדאי לדחוס ב-LZ?

In [ ]:
import math

def lz_bits_analysis(W=4096, L=256):
    """ניתוח כמות הביטים ב-LZ"""
    bits_offset = math.ceil(math.log2(W))  # ביטים להיסט
    bits_length = math.ceil(math.log2(L))  # ביטים לאורך
    bits_char = 8  # ביטים לתו (ASCII)
    
    bits_reference = bits_offset + bits_length + 1  # +1 לסימון סוג
    
    print(f"Parameters: W={W}, L={L}")
    print(f"Bits for offset: {bits_offset}")
    print(f"Bits for length: {bits_length}")
    print(f"Bits for reference: {bits_reference}")
    print(f"Bits for char: {bits_char + 1}")  # +1 לסימון סוג
    print()
    
    # מתי כדאי לדחוס?
    min_length = math.ceil(bits_reference / bits_char)
    print(f"Minimum length to compress: {min_length} characters")
    
    return min_length

lz_bits_analysis()
print()
lz_bits_analysis(W=200, L=50)